In [ ]:
import os
# Find the latest version of spark 3  from http://www.apache.org/dist/spark/

spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()
!pip install pyspark



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 222 kB in 2s (138 kB/s)
Reading package lists... Done
/content


In [ ]:
# start the spark session

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DFBasics").getOrCreate()

In [ ]:
df = spark.createDataFrame(
    [
      (0, "row1"),
      (1, "row2"),
      (2, "row3"),
    ],
    [
      "id",
      "col1"
    ]
)

In [ ]:
df.show()

+---+----+
| id|col1|
+---+----+
|  0|row1|
|  1|row2|
|  2|row3|
+---+----+



In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/v2/module_17/food.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("food.csv"), sep=",", header=True)

In [ ]:
df.show()

+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    0|
|  sushi|   12|
|chinese|   10|
+-------+-----+



In [ ]:
df.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: string (nullable = true)



In [ ]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
# Next we need to create the list of struct fields
schema = [
    StructField(
        "food",
        StringType(),
        True),
    StructField(
        "price",
        IntegerType(),
        True),
        ]
schema

[StructField('food', StringType(), True),
 StructField('price', IntegerType(), True)]

In [ ]:
# Pass in our fields
final = StructType(fields=schema)
final

StructType([StructField('food', StringType(), True), StructField('price', IntegerType(), True)])

In [ ]:
# Read our data with our new schema
dataframe = spark.read.csv(
    SparkFiles.get("food.csv"), schema=final, sep=",", header=True
    )
dataframe.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: integer (nullable = true)



In [ ]:
# Add new column
dataframe.withColumn('newprice', dataframe['price']).show()
# Update column name
dataframe.withColumnRenamed('price','newerprice').show()
# Double the price
dataframe.withColumn('doubleprice',dataframe['price']*2).show()
# Add a dollar to the price
dataframe.withColumn('add_one_dollar',dataframe['price']+1).show()
# Half the price
dataframe.withColumn('half_price',dataframe['price']/2).show()

+-------+-----+--------+
|   food|price|newprice|
+-------+-----+--------+
|  pizza|    0|       0|
|  sushi|   12|      12|
|chinese|   10|      10|
+-------+-----+--------+

+-------+----------+
|   food|newerprice|
+-------+----------+
|  pizza|         0|
|  sushi|        12|
|chinese|        10|
+-------+----------+

+-------+-----+-----------+
|   food|price|doubleprice|
+-------+-----+-----------+
|  pizza|    0|          0|
|  sushi|   12|         24|
|chinese|   10|         20|
+-------+-----+-----------+

+-------+-----+--------------+
|   food|price|add_one_dollar|
+-------+-----+--------------+
|  pizza|    0|             1|
|  sushi|   12|            13|
|chinese|   10|            11|
+-------+-----+--------------+

+-------+-----+----------+
|   food|price|half_price|
+-------+-----+----------+
|  pizza|    0|       0.0|
|  sushi|   12|       6.0|
|chinese|   10|       5.0|
+-------+-----+----------+



Functions

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/v2/module_17/wine.csv"
spark.sparkContext.addFile(url)
wine_df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)

# Show DataFrame
wine_df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [ ]:
# Order a DataFrame by ascending values
wine_df.orderBy(wine_df["points"].desc()).show(5)

+-------+--------------------+--------------------+------+-----+----------+------------+--------+--------------------+--------------------+
|country|         description|         designation|points|price|  province|    region_1|region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+------------+--------+--------------------+--------------------+
|     US|This is an absolu...|           IX Estate|    99|  290|California| Napa Valley|    Napa|           Red Blend|              Colgin|
|  Italy|Here's a ���wow�۝...|            Messorio|    99|  320|   Tuscany|     Toscana|    null|              Merlot|        Le Macchiole|
| France|98-100 Barrel sam...|       Barrel sample|    99| null|  Bordeaux|    Pauillac|    null|Bordeaux-style Re...|Ch̢teau Pontet-Canet|
|     US|A stupendous Pino...|Precious Mountain...|    99|   94|California|Sonoma Coast|  Sonoma|          Pinot Noir|     Williams Selyem|
|     US|There are i

In [ ]:
# average
from pyspark.sql.functions import avg

wine_df.select(avg("points")).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [ ]:
# filter
wine_df.filter("price<20").show(5)

+--------+--------------------+-----------+------+-----+----------+--------------------+-----------------+--------------+--------------------+
| country|         description|designation|points|price|  province|            region_1|         region_2|       variety|              winery|
+--------+--------------------+-----------+------+-----+----------+--------------------+-----------------+--------------+--------------------+
|Bulgaria|This Bulgarian Ma...|    Bergul̩|    90|   15|  Bulgaria|                null|             null|        Mavrud|        Villa Melnik|
|   Spain|Earthy plum and c...|     Amandi|    90|   17|   Galicia|       Ribeira Sacra|             null|       Menc�_a|      Don Bernardino|
|      US|There's a lot to ...|       null|    90|   18|California|Russian River Valley|           Sonoma|    Chardonnay|            De Loach|
|      US|Massively fruity,...|       null|    91|   19|    Oregon|   Willamette Valley|Willamette Valley|    Pinot Gris|   Trinity Vineyards|

In [ ]:
# select certain columns on filter
wine_df.filter("price<20").select(['points','country','winery','price']).show(5)

+------+--------+--------------------+-----+
|points| country|              winery|price|
+------+--------+--------------------+-----+
|    90|Bulgaria|        Villa Melnik|   15|
|    90|   Spain|      Don Bernardino|   17|
|    90|      US|            De Loach|   18|
|    91|      US|   Trinity Vineyards|   19|
|    91|Portugal|Adega Cooperativa...|   15|
+------+--------+--------------------+-----+
only showing top 5 rows



In [ ]:
# Filter
wine_df.filter("price<20").show(5)
# Filter by price on certain columns
wine_df.filter("price<20").select(['points','country', 'winery','price']).show(5)

# Filter on exact state
wine_df.filter(wine_df["country"] == "US").show()

+--------+--------------------+-----------+------+-----+----------+--------------------+-----------------+--------------+--------------------+
| country|         description|designation|points|price|  province|            region_1|         region_2|       variety|              winery|
+--------+--------------------+-----------+------+-----+----------+--------------------+-----------------+--------------+--------------------+
|Bulgaria|This Bulgarian Ma...|    Bergul̩|    90|   15|  Bulgaria|                null|             null|        Mavrud|        Villa Melnik|
|   Spain|Earthy plum and c...|     Amandi|    90|   17|   Galicia|       Ribeira Sacra|             null|       Menc�_a|      Don Bernardino|
|      US|There's a lot to ...|       null|    90|   18|California|Russian River Valley|           Sonoma|    Chardonnay|            De Loach|
|      US|Massively fruity,...|       null|    91|   19|    Oregon|   Willamette Valley|Willamette Valley|    Pinot Gris|   Trinity Vineyards|

In [ ]:
wine_df.filter("price>15").filter(wine_df['province']=='California').show()

+-------+--------------------+--------------------+------+-----+----------+--------------------+-------------+------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|     region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+-------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|         Napa|Cabernet Sauvignon|               Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|       Sonoma|   Sauvignon Blanc|            Macauley|
|     US|The producer sour...|Gap's Crown Vineyard|    95|   60|California|        Sonoma Coast|       Sonoma|        Pinot Noir|           Blue Farm|
|     US|This blockbuster,...|     Rainin Vineyard|    95|  325|California|Diamond Mountain ..